In [20]:
import torch

In [21]:
from model import GptLanguageModel
from common import GptConfig

In [22]:
hyperparameters = GptConfig()

In [27]:
model = GptLanguageModel(hyperparameters)
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(model.device)

In [28]:
tokenizer = m.tokenizer

In [33]:
start_str = """
Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because
it couldn’t move. Every day, it would say
"""
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)

In [34]:
print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))


Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. It was sad because
it couldn’t move. Every day, it would say "If you may we forget this door'.

One day, the pumpkin was a kind lady who should do it. She lived in the pumpkin alone. Soon, the egg started to rot. It was here.

One day, there was a little, and even clearer were home in no garden. The old castle was very unhappy because it was dangerous.

They arrived in the forest and had an idea. They all went in the ant and hugged each other's heart and said to him.

Since then, something brave enough of the star in town was embarrassment, and his fear. But the pumpkin caused her dream. He needed to find the mole soon before the garden happened and couldn't move from all the mystery.

Days wasn't listen to rest of the palace he'd found a cozy home in the shadows. Then the enjoyed the sweet egg light and never take it in the dark day. 

And, the baby friend were now the story of all the marvelous of o